## Load ZF affinity and ZF pair data

In [1]:
infile = open('ZifRC_data.txt','r')
line = infile.readline()
#creating dictionaries that map zf names to their sequences, codons, and scores
zf_seq = {}
zf_codon = {}
zf_score = {}
while line:
    data = line.split('\t')
    zf_seq[data[0]] = data[1]
    zf_codon[data[0]] = data[2]
    zf_score[data[0]] = float(data[3])
    line = infile.readline()
infile.close()

In [5]:
#creating dictionary that maps ZF names to a list of all ZFs that can follow it
zf_transitions = {}
infile = open('MARIA_transitions.txt','r') #formatted with each line as "zf1_name\tzf2_name"
line = infile.readline()
while line:
    data = line.split('\t')
    zf1 = data[0]
    zf2 = data[1][:-1]
    if zf1 in zf_transitions:
        zf_transitions[zf1].append(zf2)
    else:
        zf_transitions[zf1] = [zf2]
    line = infile.readline()
infile.close()

## Generate data sets collating DNA-binding and MHC results

In [24]:
#inverting zf -> codon dictionary to get all zfs that bind a given codon
codon_zfs = {}
for zf in zf_codon.keys():
    codon = zf_codon[zf]
    if codon in codon_zfs.keys():
        codon_zfs[codon].append(zf)
    else:
        codon_zfs[codon]=[zf]
        
#creating a list of all possible trinucleotides (64)
possible_codons = set()
for n1 in ['A', 'G', 'C','T']:
    for n2 in ['A', 'G', 'C','T']:
        for n3 in ['A', 'G', 'C','T']:
            possible_codons.add(n1+n2+n3)
#creating list of nucleotides no zf in the set binds to
missing_codons = possible_codons - codon_zfs.keys()

for codon in missing_codons:
    codon_zfs[codon] = []
    
codon_transitions = {}
for zf1 in zf_transitions.keys():
    if zf1 in zf_codon:
        codon2 = zf_codon[zf1]
        for zf2 in zf_transitions[zf1]:
            if zf2 in zf_codon:
                codon1 = zf_codon[zf2]
                if codon1 in codon_transitions.keys():
                    if not (codon2 in codon_transitions[codon1]):
                        codon_transitions[codon1].append(codon2)
                else:
                    codon_transitions[codon1] = [codon2]
                
for codon in possible_codons:
    if not codon in codon_transitions.keys():
        codon_transitions[codon] = []
    
zf_possible_per_codon = {}
for zf1 in zf_transitions.keys():
    for codon in possible_codons:
        name = zf1 + codon
        zf_list = []
        for zf2 in zf_transitions[zf1]:
            if zf2 in zf_codon and zf_codon[zf2] == codon:
                zf_list.append(zf2)
        zf_possible_per_codon[name] = zf_list

## Define functions for ZF targeting

In [13]:
def split_seq(sequence):
    if len(sequence) < 3:
        return []
    if len(sequence) % 3 != 0:
        sequence = sequence[:-(len(sequence) % 3)]
    temp_seq =sequence
    output = []
    while temp_seq:
        output.append(temp_seq[-3:])
        temp_seq = temp_seq[:-3]
    return output

In [11]:
class zf_tree:
    #object represents a set of zinc fingers that bind to the same DNA sequence. Each node represents a given zinc finger domain,
    #while the children are themselves trees which are the zfs that can directly follow the parent tree's ZF
    def __init__(self, zf):
        self.zf = zf
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def return_arrays(self):
        if self.children:
            output = []
            for child in self.children:
                for array in child.return_arrays():
                    output.append([self.zf] + array)
            return output
        else:
            return [[self.zf]]
    def return_best_array(self,zf_score):
        if self.children:
            best_score = 0
            best_child = []
            for child in self.children:
                child_best = child.return_best_array()
                child_score = grade_array(child_best,zf_score)
                if child_score > best_score:
                    best_child = child_best
                    best_score = child_score
            return [self.zf] + best_child
        else:
            return[self.zf]
    def return_best_score(self,zf_score):
        if self.children:
            best_score = 0
            for child in self.children:
                this_score = child.return_best_score(zf_score)
                if this_score > best_score:
                    best_score = this_score
            return best_score + zf_score[self.zf]
        else:
            return zf_score[self.zf]
    def return_array_nums(self):
        if self.children:
            array_num = 0
            for child in self.children:
                array_num = array_num + child.return_array_nums()
            return array_num
        else:
            return 1
        

In [12]:
def seq_tree(sequence, missing_codons,codon_zfs,zf_transitions,zf_possible_per_codon,codon_transitions):
    #with seq_tree_recursive, creates a list of trees that represents all zf arrays that can bind to the input sequence. 
    #note that this only looks at the positive strand sequence; should run on reverse complement also (since the ZF can bind in either orientation)
    codon_list = split_seq(sequence)
    for i in range(len(codon_list)):
        if codon_list[i] in missing_codons: #return empty list without starting 
            return None                     #if you know there are codons no zfs bind to in the sequence
        if i < len(codon_list) - 1:
            if not (codon_list[i+1] in codon_transitions[codon_list[i]]): #or if a codon that cannot follow another appears in the sequence
                return None
    trees = []
    for zf_1 in codon_zfs[codon_list[0]]: #for every zf that can bind the first codon
        base_tree = []
        if zf_1 in zf_transitions.keys(): #and which can be followed by any other zfs
            for zf_2 in zf_possible_per_codon[zf_1+codon_list[1]]: #then for every zf that can follow the first zf and bind the second codon
                tree = seq_tree_recursive(codon_list[1:],zf_2, missing_codons,codon_zfs,zf_transitions,zf_possible_per_codon) #create the tree with that zf at the root
                if tree: #if there's a nonempty return, then some zf tree can bind the sequence, and should be added to the tree
                    if not base_tree:
                        base_tree = zf_tree(zf_1)
                    base_tree.add_child(tree)
        if base_tree:
            trees.append(base_tree)
    return trees


def seq_tree_recursive(codon_list,zf_1, missing_codons,codon_zfs,zf_transitions,zf_possible_per_codon,skip_list = {}):
    if len(codon_list) == 1: #if only one codon, return the finger you're starting with since you don't need to add anything else
        return zf_tree(zf_1)
    # if we are on ZF_A, which can be followed by either ZF_B1 or ZF_B2
    # but both ZF_B1 and ZF_B2 can be followed by ZF_C
    # then the tree beyond ZF_C will be the same in both cases and we don't need to generate it twice
    # so for every such ZF_C we add it and the child with it as the zf to next_skip
    # and pass that to every subsequent recursive call (in this step) as skip_list
    next_skip = {} 
    if zf_1 in zf_transitions.keys(): #if no zfs can follow the one you're starting with then this is skipped and None is returned
        base_tree = []
        for zf_2 in zf_possible_per_codon[zf_1 + codon_list[1]]: #for every zf that can follow the current zf and bind the next codon
            if not zf_2 in skip_list.keys(): # if we haven't seen it already in this position from the previous step
                tree = seq_tree_recursive(codon_list[1:],zf_2, missing_codons,codon_zfs,zf_transitions,zf_possible_per_codon,next_skip) #create the tree with that zf at the root
                if tree: 
                    for child in tree.children: #note that we've seen all the possible ZFs that can follow this ZF and their subsequent branches
                        next_skip[child.zf] = child #add them so as not to redo that work
            else: # if we have seen this ZF in the next position
                tree = skip_list[zf_2] #then we know the child that has to follow it
            if tree: #if there's a nonempty return, then some zf tree can bind the rest of sequence, and should be added to the tree
                if not base_tree:
                    base_tree = zf_tree(zf_1)
                base_tree.add_child(tree)
        if base_tree:
            return base_tree         
            
def num_arrays(tree_list):
    output = 0
    for tree in tree_list:
        output = output + tree.return_array_nums()
    return(output)

def grade_array(array,zf_score):
    score = 0
    for zf in array:
        score = score + zf_score[zf]
    return score

def ranked_arrays(tree_list,zf_score):
    all_arrays = []
    for tree in tree_list:
        all_arrays += tree.return_arrays()
    scored_arrays = [[array,grade_array(array,zf_score)] for array in all_arrays]
    output = sorted(scored_arrays, key = lambda x: x[1],reverse=True)
    return output

def get_scored_arrays(tree_list):
    arrays = []
    for tree in tree_list:
        arrays += tree.return_arrays()
    scored_arrays = [[array,grade_array(array,zf_score)] for array in arrays]
    return scored_arrays

def get_best_array(tree_list,zf_score):
    best_score = 0
    best_array =[]
    for tree in tree_list:
        this_array = tree.return_best_array(zf_score)
        this_score = grade_array(this_array,zf_score)
        if this_score > best_score:
            best_score = this_score
            best_array = this_array
    return best_array

def write_array(array_list):
    output = ''
    for array in array_list:
        output = output + array + '-'
    return output[:-1]
def array_sequence(array):
    output = zf_seq[array[0]]
    for zf in array[1:]:
        output = output + 'TGERP' + zf_seq[zf]
    return output     

def get_best_score(tree_list,zf_score):
    best_score = 0
    for tree in tree_list:
        this_score = tree.return_best_score(zf_score)
        if this_score > best_score:
            best_score = this_score
    return best_score

In [14]:
# Gets the DNA base complement of a base
def complement_DNA_bp(base):
    switcher = {
        'A': "T",
        'T': "A",
        'G': 'C',
        'C': 'G',
        'N': 'N'
    }
    return switcher.get(base)

# Returns complement of a DNA sequence in 5' to 3' direction
# assuming 5' to 3' input
def complement_DNA(seq):
    complement = ""
    for i in range(len(seq)-1, -1,-1):
        complement+=complement_DNA_bp(seq[i])
    return complement

## Get number of possible arrays and maximum scores for sites in UTRN and SCN1A promoters

In [15]:
utro_seq = 'TTCACTCTGGAGCGCGCGCCCCAGGCCAGCCAAGCGCCGAGCCGGGCTGCTGCGGGCTGGGAGGGCGCGCAGGGCCGGCGCTGATTGACGGGGCGCGCAGTCAGGTGACTTGGGGCGCCAAGTTCCCGACGCGGTGGCCGCGGTGACCGCCGAGGCCCGGCAGACGCTGACCCGGGAACGTAGTGGGGCTGATCTTCCGGAACAAAGTTGCTGGGCCGGCGGCGGCGGGGCGAGAGCGCCGAGGGGGAGCCGGAGCGCTGCAGAGGCGCGGGCCGGAGGGCTGGCGCTGATCTGCACCCTTCTCATCTGGAGAGCGGACCCCTGGCTGCCCGGAGGCGAGCCCCTTCCCGGGGGGTGGGGGCGGCAACGCGCGACCCAGCGGTCCTGCGCCCCACCCTCCCTCCTCCGCCTCCAGCGCTCGGCTCCAACAAAGGGGCAGGCCCGCAGCGGGGAGGAGGAGGAGGAGCCGCCGAAGGAGCGAGCCTCTCTCGCGCACAAAGTTGTGGAGTCGTTTTTCCTCGGAGCAGGGAAGCGGGCAGCAGCAGCCGGCCGCGGGCTTTCTCCCGCCGAGGGGCGAGGAGGAGCCTCTGGCTCCAGAAG'

In [16]:
scn1a_seq = 'AACTATTTGATTGGTCATTAAGGTTGGTCACTGCAAACTATCCTGACTATCTTGGCCAGGAATCACTTGTCAGACCTAGAGGGTTAGATTTTATGTCCTCTCAGAGACAATTCCTTTTCCCCTCATCCTTGTTAAATGAGCTATCCTTTGCCTTTATACGCACAGTCTCCATCTTCTGGGTGTGTCACAGACTGAATAACTCATTAGTGATATAGTGTTAATTAGCCTAATGTACTTTGATTAAGGCCATATTTGAACCACTTTTAAAACTCATGGCACAGTTCCTGTATCAGTAAAGCACAAGAATTAATAAATAAGTGATGCTTAACTAAACTCAACACAGAAACCATTTGTGTTAAAATTTTTTCTTTAGAAATCACCTTTCAATTTAAGGAGAAAACAGACTTTAAATCCTCTAGCTCATGTTTCATGACAAGAATTTATTTATATTAACATCTCTTAGTCCACTCTTTAAAATATCTGTATTCCTTTTATTTTAGGAATTTCATATGCAGAATAAATGGTAATTAAAATGTGCAGGATGACAAGATGGAGCAAACAGTGCTTGTACCACCAGGACCTGACAGCTTCAACTTCTTC'

In [29]:
def promoter_site_nums_and_scores(promoter_seq, zf_num,missing_codons,codon_zfs,zf_transitions,zf_possible_per_codon,codon_transitions,zf_score):
    site_length = 3 * zf_num
    site_num = 0 
    reverse_strand = complement_DNA(promoter_seq)
    strands = (promoter_seq,reverse_strand)
    site_sequences = []
    site_nums = []
    site_max_scores = []
    for strand in strands:
        for i in range(len(strand)-site_length):
            site = strand[i:i+site_length].upper()
            site_tree = seq_tree(site,missing_codons,codon_zfs,zf_transitions,zf_possible_per_codon,codon_transitions)
            if site_tree:
                site_sequences.append(site)
                site_nums.append(num_arrays(site_tree))
                site_max_scores.append(get_best_score(site_tree,zf_score))
    return site_sequences, site_nums, site_max_scores

In [30]:
utrn_sites, utrn_nums, utrn_scores = promoter_site_nums_and_scores(utro_seq, 8,missing_codons,
                            codon_zfs,zf_transitions, zf_possible_per_codon,codon_transitions,zf_score)

In [31]:
outfile = open('utrn_6b_data.txt','w')
for i in range(len(utrn_sites)):
    outfile.write('{}\t{}\t{}\n'.format(utrn_sites[i], utrn_nums[i], utrn_scores[i]))
outfile.close()

In [32]:
scn1a_sites, scn1a_nums, scn1a_scores = promoter_site_nums_and_scores(scn1a_seq, 8,missing_codons,
                            codon_zfs,zf_transitions, zf_possible_per_codon,codon_transitions,zf_score)

In [34]:
outfile = open('scn1a_6b_data.txt','w')
for i in range(len(scn1a_sites)):
    outfile.write('{}\t{}\t{}\n'.format(scn1a_sites[i], scn1a_nums[i], scn1a_scores[i]))
outfile.close()